### Clear memory

In [2]:
%reset -f
import gc
gc.collect()

0

### Import

In [ ]:
import pandas as pd
import sys, os
from sentence_transformers import SentenceTransformer
from chromadb import PersistentClient
from chromadb.config import Settings
from chromadb.api.models.Collection import Collection
from typing import Any, Dict
from anthropic import Anthropic
from dotenv import load_dotenv
from tqdm.auto import tqdm
import ragas
from datasets import Dataset
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [5]:
test_dataset = [
    {
        'id': 0,
        'question': 'How do I make a tasty pizza?',
        'reference_answer': 'This question is not related to AWS documentation.',
        'page_numbers': [],
        'category': 'irrelevant',
    },
    {
        'id': 1,
        'question': 'Who is the current president of USA?',
        'reference_answer': 'This question is not related to AWS documentation.',
        'page_numbers': [],
        'category': 'irrelevant',
    },
    {
        'id': 2,
        'question': 'What is the capital of Germany?',
        'reference_answer': 'This question is not related to AWS documentation.',
        'page_numbers': [],
        'category': 'irrelevant',
    },
    {
        'id': 3,
        'question': 'If I do not have an AWS account, what do I do?',
        'reference_answer': 'To sign up for an AWS account open https://portal.aws.amazon.com/billing/signup and follow the online instructions.',
        'page_numbers': [10],
        'category': 'relevant',
    },
    {
        'id': 4,
        'question': 'What if I want to allow people outside of my AWS account to access my AWS resources?',
        'reference_answer': 'You can create a role that users in other accounts or people outside of your organization can use to access your resources. You can specify who is trusted to assume the role. For services that support resource-based policies or access control lists (ACLs), you can use those policies to grant people access to your resources.',
        'page_numbers': [114],
        'category': 'relevant',
    },
    {
        'id': 5,
        'question': 'What is the AWS Toolkit for Microsoft Azure DevOps?',
        'reference_answer': 'AWS Toolkit for Microsoft Azure DevOps is an extension for Microsoft Azure DevOps that contains tasks you can use in build and release definitions to interact with AWS services. It is available through the Visual Studio Marketplace.',
        'page_numbers': [7],
        'category': 'relevant',
    },
    {
        'id': 6,
        'question': 'Where can I install the AWS Toolkit for Azure DevOps extension?',
        'reference_answer': 'You can install the AWS Toolkit for Azure DevOps extension from the Extensions for Azure DevOps Visual Studio Marketplace. The direct link is https://marketplace.visualstudio.com/items?itemName=AmazonWebServices.aws-vsts-tools',
        'page_numbers': [12],
        'category': 'relevant',
    },
    {
        'id': 7,
        'question': 'How many ways can I supply AWS credentials to tasks?',
        'reference_answer': 'You can supply credentials in four ways: using a service connection, through named variables in your build (AWS.AccessKeyID, AWS.SecretAccessKey, AWS.SessionToken), through standard AWS environment variables in the build agent process, or with Amazon EC2 build agents using instance metadata.',
        'page_numbers': [13, 14, 15],
        'category': 'relevant',
    },
    {
        'id': 8,
        'question': 'What permissions does AWS Send SNS or SQS Message task require?',
        'reference_answer': 'This task requires permissions to call the following AWS service APIs (depending on selected task options, not all APIs may be used): sns:GetTopicAttributes, sns:Publish, sqs:GetQueueAttributes and sqs:SendMessage.',
        'page_numbers': [95],
        'category': 'relevant',
    },
    {
        'id': 9,
        'question': 'Which deployment types does the AWS Lambda .NET Core task support?',
        'reference_answer': 'The task supports two deployment types: Function (deploys a single function or creates a package zip file) and Serverless Application (performs deployment using AWS CloudFormation for multiple functions or builds and uploads to S3).',
        'page_numbers': [76,77],
        'category': 'relevant',
    },
    {
        'id': 10,
        'question': 'What structure does AWS CLI use?',
        'reference_answer': 'The AWS CLI uses a multipart structure on the command line: <command> <subcommand> [options and parameters].',
        'page_numbers': [33,34],
        'category': 'relevant',
    },
    {
        'id': 11,
        'question': 'Can I create an S3 bucket automatically with the S3 Upload task?',
        'reference_answer': 'Yes, you can select the checkbox "Create S3 bucket if it does not exist" and the task will attempt to create the bucket if it does not exist. Note that bucket names must be globally unique.',
        'page_numbers': [79],
        'category': 'relevant',
    },
    {
        'id': 12,
        'question': 'What is the maximum timeout value for the AWS CodeDeploy deployment task?',
        'reference_answer': 'The default maximum timeout is 60 minutes.',
        'page_numbers': [56],
        'category': 'relevant',
    },
    {
        'id': 13,
        'question': 'What AWS Shell Script task is used for?',
        'reference_answer': 'Runs a shell script in Bash, setting AWS credentials and Region information into the shell environment using the standard environment keys AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_SESSION_TOKEN and AWS_REGION.',
        'page_numbers': [39],
        'category': 'relevant',
    },
    {
        'id': 14,
        'question': 'Which task should I use to push a Docker image to Amazon ECR?',
        'reference_answer': 'You should use the Amazon ECR Push task (also known as Amazon Elastic Container Registry Push Image Task). This task pushes a Docker image identified by name with optional tag, or image ID to ECR.',
        'page_numbers': [51],
        'category': 'relevant',
    },
]


In [6]:
test_df = pd.DataFrame(test_dataset)

print(f'✅ Created {len(test_df)} test questions')
print(f'📊 Relevant: {(test_df["category"] == "relevant").sum()}')
print(f'📊 Irrelevant: {(test_df["category"] == "irrelevant").sum()}')

test_df.head()

✅ Created 15 test questions
📊 Relevant: 12
📊 Irrelevant: 3


,id,question,reference_answer,page_numbers,category
0,0,How do I make a tasty pizza?,This question is not related to AWS documentat...,[],irrelevant
1,1,Who is the current president of USA?,This question is not related to AWS documentat...,[],irrelevant
2,2,What is the capital of Germany?,This question is not related to AWS documentat...,[],irrelevant
3,3,"If I do not have an AWS account, what do I do?",To sign up for an AWS account open https://por...,[10],relevant
4,4,What if I want to allow people outside of my A...,You can create a role that users in other acco...,[114],relevant


In [ ]:
# Run all test questions through RAG pipeline
results = []

for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc='Running RAG'):
    question = row['question']

    # Get RAG response
    rag_result = rag_pipeline(
        query=question,
        embedding_model=model,
        collection=collection,
        n_results=5,
    )

    # Extract retrieved contexts
    contexts = [doc for doc in rag_result['search_results']['documents'][0]]

    results.append({
        'question': question,
        'answer': rag_result['answer'],
        'contexts': contexts,
        'ground_truth': row['reference_answer'],
    })

# Convert to DataFrame
eval_df = pd.DataFrame(results)

In [ ]:


# Convert to HuggingFace Dataset format (required by RAGAS)
ragas_dataset = Dataset.from_pandas(eval_df)